## 混合混触危険性予測（2024/2/18作成）

評価結果を100点満点に変更（Analyze.ipynbはこの結果をもとに作成）

In [1]:
import pandas as pd
import openai
from tqdm import tqdm
from src.utils.GPT import json_generate
from src.qra_gen import generate_reason_and_predict_mixture

In [2]:
#api_key
openai.api_key = "api_key"

In [3]:
#laod dataset as dict
csv_path="dataset/Updated_Cameo_reactivity_sorted.csv"
df=pd.read_csv(csv_path)
df = df.iloc[:,:3]
chemical_records=df.to_dict(orient = "records")
chemical_records

[{'substance1': 'CARBON, ACTIVATED',
  'substance2': 'FERRIC SULFATE',
  'result': 'Compatible'},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'FERRIC NITRATE',
  'result': 'Incompatible'},
 {'substance1': 'CARBON, ACTIVATED',
  'substance2': 'METHANE',
  'result': 'Compatible'},
 {'substance1': 'AIR, COMPRESSED',
  'substance2': 'WATER',
  'result': 'Compatible'},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'WATER',
  'result': 'Incompatible'},
 {'substance1': 'FERRIC SULFATE',
  'substance2': 'IRON OXIDE, SPENT',
  'result': 'Incompatible'},
 {'substance1': 'FERRIC NITRATE',
  'substance2': 'IRON, [POWDERED]',
  'result': 'Incompatible'},
 {'substance1': 'ETHANOL',
  'substance2': 'PROPIONALDEHYDE',
  'result': 'Caution'},
 {'substance1': 'CARBON, ACTIVATED',
  'substance2': 'FERRIC CHLORIDE',
  'result': 'Compatible'},
 {'substance1': 'IRON OXIDE, SPENT',
  'substance2': 'WATER',
  'result': 'Incompatible'},
 {'substance1': 'PROPIONALDEHYDE',
  'substanc

In [14]:
#このプロンプトを変更

# 質問文を生成するための指示
#qa_gen_command = """
#Provide the quantitative reasons within 300 words so that a scientist, who does not know the compatibility or reactivity of a mixture, can predict these outcomes.
#We must quantitatively consider how the combination of substances influences their compatibility or reactivity.
#I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler compounds, contributing to higher reactivity. (qualitative discussion is practically meaningless!!!)
## Therefore, the mixture is highly reactive. (Never include the answer in the reason!!).

#Good example reasons
## Mixing sodium with water generates a significant amount of heat and hydrogen gas due to the chemical reaction, which can lead to an explosive reaction under certain conditions.
## The combination of acetone and bleach creates chloroform and other hazardous compounds due to the chemical reaction between acetone's methyl groups and sodium hypochlorite in bleach.

#Output: Reason key
#"""

qa_gen_command = """
Provide detailed and quantitative reasons within 300 words to accurately predict the compatibility or reactivity of a mixture. Focus on the specific interactions between substances and their measurable effects on the mixture’s safety. Avoid making broad generalizations and ensure your reasoning is grounded in scientific principles and data.

#Bad example reasons
## Its molecular weight compared to simpler compounds, contributing to higher reactivity. (This is too vague and does not consider specific interactions.)
## Therefore, the mixture is highly reactive. (This conclusion lacks supporting quantitative data.)

#Good example reasons
## Mixing sodium with water increases the temperature of the solution up to 100°C within 30 seconds due to an exothermic reaction, releasing 56.6 kJ/mol of energy. This rapid energy release can cause boiling and splattering, posing a burn hazard.
## The combination of acetone and bleach at a ratio of 1:1 by volume generates chloroform, with a reaction yield of 20% based on acetone. The presence of chloroform, beyond a concentration of 0.002 ppm in air, can be neurotoxic and carcinogenic, indicating a specific health risk.

#Output: Reason key
Consider not only the potential for a reaction but also the conditions under which the reaction is safe or hazardous. Include considerations such as concentration thresholds, reaction rates, and environmental factors that mitigate or exacerbate risks.
"""

# 分子構造から物性を予測するための指示
#predict_command = """
#Predict the compatibility or reactivity of the following mixture.
#Consider the quantitative reasons provided and, in any case, output a prediction about the mixture's safety or potential hazards.

#Output: Result key
#"""

#predict_command = """
#Predict the compatibility or reactivity of the following mixture, using only one of the following outcomes: Compatible, Incompatible, Caution. 
#Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards.

#Output: Result key should be one of [Compatible, Incompatible, Caution].
#"""

predict_command = """
Predict the level of danger of the following mixture on a scale of 0 to 100, where 0 indicates no danger at all and 100 represents the highest level of danger. 
Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards. Provide your prediction as a numerical value reflecting the mixture's level of danger.

#Output: Result key should be a numerical value between 0 and 100.
"""

#use GPT3.5 completion
llm_ask_func=json_generate

#if gen_reason=True, generate reason and predict
#if gen_reason=False, only predict without reason (for control)
gen_reason=True
#gen_reason=False

#n回のpredictionを行う
n_trials=3

In [15]:


completed_records = []
for record in tqdm(chemical_records):  # tqdmは進捗状況を表示
    try:
        print("Hello")
            # generate_reason_and_predict関数により、理由生成と予測実行
        completed_record = generate_reason_and_predict_mixture(
            substance1=record["substance1"],
            substance2=record["substance2"],
            result=record["result"],  # resultは混合物の危険性判定の結果
            qa_gen_command=qa_gen_command,  # 理由を尋ねるための指示
            predict_command=predict_command,  # 予測を行うための指示
            llm_ask_func=json_generate,  # LLM関数
    #            smiles=record.get("smiles", ""),  # SMILES情報（存在する場合）
            gen_reason=gen_reason,  # 理由を生成するかどうか
            n_trials=n_trials  # 予測を試みる回数
        )
#        print(completed_record)
    except Exception as e:
        print("negative")
#        traceback.print_exc()
        print(e)  # 例外が発生した場合にエラーを表示
        completed_record = record  # 例外が発生した場合は元のレコードを使用

    completed_records.append(completed_record)  # 処理されたレコードをリストに追加


  0%|                                                                                           | 0/91 [00:00<?, ?it/s]

Hello
{'Result': 20}
{'Result': 10}


  1%|▉                                                                                  | 1/91 [00:05<07:33,  5.04s/it]

{'Result': 20}
Hello
{'Result': 95}
{'Result': 95}


  2%|█▊                                                                                 | 2/91 [00:11<08:27,  5.70s/it]

...

negative
Expecting value: line 1132 column 2 (char 1237)
Hello
{'Result': 60}
{'Result': 60}


 99%|█████████████████████████████████████████████████████████████████████████████████ | 90/91 [06:12<00:16, 16.38s/it]

{'Result': 60}
Hello
{'Result': 10}
{'Result': 10}


100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [06:16<00:00,  4.13s/it]

{'Result': '10'}


In [11]:
completed_records

[{'substance1': 'CARBON, ACTIVATED',
  'substance2': 'FERRIC SULFATE',
  'result': 'Compatible',
  'generated_reason': 'The combination of activated carbon and ferric sulfate can lead to exothermic reactions, as ferric sulfate is a strong oxidizing agent, and the activated carbon can act as a fuel. This can result in the release of heat and potentially hazardous conditions. Therefore, the mixture of activated carbon and ferric sulfate is incompatible and should be handled with care.',
  'predicted_results': [85, 85, 85]},
 {'substance1': 'ALUMINUM ALKYL HYDRIDES',
  'substance2': 'FERRIC NITRATE',
  'result': 'Incompatible',
  'generated_reason': 'The combination of aluminum alkyl hydrides and ferric nitrate can lead to a violent reaction due to the potential for ligand exchange between the alkyl groups and the nitrate ligands. This could result in the release of flammable or toxic gases, as well as the formation of unstable intermediates, posing a significant safety hazard.',
  'predi

In [16]:
#save
import datetime
import json

study_name="mixture_practice_w_reason_numeric"
now=datetime.datetime.now()
now_str=now.strftime("%Y%m%d_%H%M%S")
save_path=f"results/{study_name}_{now_str}.json"
with open(save_path, 'w', encoding = "utf-8") as f:
    json.dump(completed_records, f, indent=4, ensure_ascii=False)